In [1]:
import numpy as np
import pandas as pd
import string
import itertools

import participation_utils as plb

from importlib import reload

In [2]:
filename = '../data/Plt_sci_publications_updated_5.03.csv'

data = pd.read_csv(filename, usecols=range(36))
#data = data.iloc[:,:36]
data = data.drop_duplicates(subset=list(data.columns[:3]) + ['DOI'])
print(data.shape)

/home/ejam/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (24,27,28,31,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(292434, 36)


In [3]:
filename = '../data/Plt_sci_publications_updated_3.22.csv'

data2 = pd.read_csv(filename, usecols=range(36))
data2 = data2.drop_duplicates(subset=list(data2.columns[:3]) + ['DOI'])
print(data.shape)

(292434, 36)


In [4]:
namemissing = data2[data2['Article Title'] == '#NAME?']
dois = namemissing['DOI']
for i in range(len(dois)):
    title = data[data['DOI'] == dois.iloc[i]]['Article Title'].values[0]
    data2.loc[dois.index[i], 'Article Title'] = title
    print(i, title)

0 Epidemiology of Wheat streak mosaic virus in wheat in a Mediterranean-type environment
1 a-Tetralonyl Glucosides from the Green Walnut Husks of Juglans mandshurica and Their Antiproliferative Effects
2 b-Aminobutyric acid increases abscisic acid accumulation and desiccation tolerance and decreases water use but fails to improve grain yield in two spring wheat cultivars under soil drying
3 b-glucan content in a panel of Triticum and Aegilops genotypes


In [5]:
DOI = data['DOI']
DOI2 = data2['DOI']

for i in range(len(DOI)):
    doi = DOI.iloc[i]
    doi2 = DOI2.iloc[i]
    if not pd.isnull(doi):
        if doi != doi2:
            print('lol')
            break

In [6]:
fields = ['Author Full Names', 'Article Title', 'Author Keywords', 'Keywords Plus', 'Abstract']
for field in fields:
    data[field] = data2[field]

In [7]:
print(data.shape)

(292434, 36)


In [149]:
columns = list(data.columns)
subcolumns = columns[:1] + columns[2:6] + ['DOI']
nsubcolumns = [columns[1]] + columns[6:29] + columns[30:]

merge = data.merge(data2, how='outer', on=subcolumns, indicator=True).copy()
lidx = merge[merge['_merge'] == 'left_only'].index
print(merge.shape)

(296734, 67)


In [150]:
for xx in ['both', 'left_only', 'right_only']:
    foo = merge[merge['_merge'] == xx]
    print(xx, foo.shape, sep='\t')

both	(296718, 67)
left_only	(8, 67)
right_only	(8, 67)


In [151]:
left = merge[merge['_merge'] == 'left_only']
left.index

Int64Index([6057, 21599, 22571, 40869, 60426, 221360, 246850, 290771], dtype='int64')

In [166]:
i = 290771
j = -1
#for j in range(len(subcolumns)):
field = subcolumns[j]
title = merge.loc[i, field]
foo = data[data[field] == title]
bar = data2[data2[field] == title]
print(foo.index, bar.index, '\n', sep='\n')
if len(bar) > 0:
    for field in subcolumns:
        print(field, foo[field].values[0], bar[field].values[0], foo[field].values[0] == bar[field].values[0], '-----\n', sep='\n')

Int64Index([286565], dtype='int64')
Int64Index([286543], dtype='int64')


Author Full Names
Lee, Sais-Beul; Kim, Namgyu; Jo, Sumin; Hur, Yeon-Jae; Lee, Ji-Youn; Cho, Jun-Hyeon; Lee, Jong-Hee; Kang, Ju-Won; Song, You-Chun; Bombay, Maurene; Kim, Sung-Ryul; Lee, Jungkwan; Seo, Young-Su; Ko, Jong-Min; Park, Dong-Soo
Lee, Sais-Beul; Kim, Namgyu; Jo, Sumin; Hur, Yeon-Jae; Lee, Ji-Youn; Cho, Jun-Hyeon; Lee, Jong-Hee; Kang, Ju-Won; Song, You-Chun; Mumbai, Maurene; Kim, Sung-Ryul; Lee, Jungkwan; Seo, Young-Su; Ko, Jong-Min; Park, Dong-Soo
False
-----

Journal
PLANTS-BASEL
PLANTS-BASEL
True
-----

impact factor (2020)
3.935
3.935
True
-----

Author Keywords
Oryza sativa L; bakanae disease; Gibberella fujikuroi; QTL mapping-qBK1(Z)
Oryza sativa L; bakanae disease; Gibberella fujikuroi; QTL mapping-qBK1(Z)
True
-----

Keywords Plus
nan
nan
False
-----

DOI
10.3390/plants10030434
10.3390/plants10030434
True
-----



In [58]:
file2 = '../data/Plt_sci_publications_missing_records_2009_8.2.22.csv'
file2 = '../data/Plt_sci_publications_with_inferred_gender_and_journal_info_7.18.csv'
file2 = '../data/Plt_sci_publications_complete_set_with_gender_and_GDP_8.4.22.csv'
#file2 = '../data/foo.csv'
data2 = pd.read_csv(file2, index_col=False, usecols=range(52))
data2 = data2.drop_duplicates(ignore_index=True)
data2 = data2.drop_duplicates(subset=list(data.columns[:3]) + ['DOI'])
print(data2.shape)

(296425, 52)


In [59]:
cols = set(data.columns)
col2 = set(data2.columns)
col2 = sorted(list(col2 - cols))
col2

['#_samples_gender_inference',
 'Corresponding_author_last_name',
 'First_names_currated',
 'First_names_extracted',
 'GDP_2020',
 'Journal location',
 'Name of the society',
 'Open Access options available',
 'Open access fees (USD)',
 'Per_capita_income_2020',
 'Quartile',
 'Society (Yes or No)',
 'Subspecality (JCR)',
 'first and last names',
 'gender_inference_accuracy',
 'inferred_gender']

In [60]:
namemissing = data2[data2['Article Title'] == '#NAME?']
dois = namemissing['DOI']
for i in range(len(dois)):
    title = data[data['DOI'] == dois.iloc[i]]['Article Title'].values[0]
    data2.loc[dois.index[i], 'Article Title'] = title
    print(i, title)

0 Epidemiology of Wheat streak mosaic virus in wheat in a Mediterranean-type environment
1 a-Tetralonyl Glucosides from the Green Walnut Husks of Juglans mandshurica and Their Antiproliferative Effects
2 b-Aminobutyric acid increases abscisic acid accumulation and desiccation tolerance and decreases water use but fails to improve grain yield in two spring wheat cultivars under soil drying
3 b-glucan content in a panel of Triticum and Aegilops genotypes


In [62]:
columns = list(data.columns)
subcolumns = columns[:6] + ['DOI']
nsubcolumns = columns[6:29] + columns[30:]

merge = data.merge(data2, how='outer', on=subcolumns, indicator=True).copy()
lidx = merge[merge['_merge'] == 'left_only'].index
print(merge.shape)

(296447, 82)


In [63]:
for xx in ['both', 'left_only', 'right_only']:
    foo = merge[merge['_merge'] == xx]
    print(xx, foo.shape, sep='\t')

both	(292412, 82)
left_only	(22, 82)
right_only	(4013, 82)


```
both	(292412, 81)
left_only	(22, 81)
right_only	(4013, 81)
```

In [65]:
left = merge[merge['_merge'] == 'left_only']
left.index

Int64Index([  7844,   7845,   7846,  37033,  37034,  37035,  37036,  49525,
             90837,  90838,  90839, 115424, 119704, 176928, 176929, 176930,
            176931, 176932, 180456, 192843, 192844, 192845],
           dtype='int64')

In [66]:
i = 235780
j = -1
field = subcolumns[j]
title = merge.loc[i, field]
foo = data[data[field] == title]
bar = data2[data2[field] == title]
print(foo.index, bar.index, '\n', sep='\n')
if len(bar) > 0:
    for field in subcolumns:
        print(field, foo[field].values[0], bar[field].values[0], foo[field].values[0] == bar[field].values[0], '-----\n', sep='\n')

Int64Index([235854], dtype='int64')
Int64Index([235832], dtype='int64')


Author Full Names
Bhatia, Harpreet; Sharma, Yash Pal; Manhas, R. K.; Kumar, Kewal
Bhatia, Harpreet; Sharma, Yash Pal; Manhas, R. K.; Kumar, Kewal
True
-----

Article Title
Traditional phytoremedies for the treatment of menstrual disorders in district Udhampur, J&K, India
Traditional phytoremedies for the treatment of menstrual disorders in district Udhampur, J&K, India
True
-----

Journal
JOURNAL OF ETHNOPHARMACOLOGY
JOURNAL OF ETHNOPHARMACOLOGY
True
-----

impact factor (2020)
4.36
4.36
True
-----

Author Keywords
Ethnobotany; Ethnogynecological problems; Menstrual disorders; Phytoremedies; Traditional plant knowledge; Udhampur district
Ethnobotany; Ethnogynecological problems; Menstrual disorders; Phytoremedies; Traditional plant knowledge; Udhampur district
True
-----

Keywords Plus
MEDICINAL-PLANTS; KNOWLEDGE; DISEASES; TRIBES; STATE
MEDICINAL-PLANTS; KNOWLEDGE; DISEASES; TRIBES; STATE
True
-----

DOI
10.1016

In [67]:
for c in nsubcolumns:
    colnamex = c + '_x'
    colnamey = c + '_y'
    merge.loc[merge['_merge'] == 'left_only', colnamey] = merge.loc[merge['_merge'] == 'left_only', colnamex]

In [ ]:
# Check whether there are entries s.t. they have _x values but no _y values
left = merge[merge['_merge'] == 'left_only'].copy()
print(left.shape)
leftidx = set(left.index)

for c in nsubcolumns:
    colnamex = c + '_x'
    colnamey = c + '_y'
    
    foo = merge[~pd.isnull(merge[colnamex])]
    fidx = set(foo[pd.isnull(merge[colnamey])].index)
    
    print(c, '\t', fidx.issubset(leftidx))
    
# Turns out that the only such entries are within the `left_only` dataframe

In [71]:
plft = ['Addresses--all authors','Nations--all authors','Corresponding author Addresses', 'Abstract',
        'Corresponding author Nation','Corresponding author Continent','Corresponding author global designation']

for c in plft:
    colnamex = c + '_x'
    colnamey = c + '_y'
    merge.loc[merge['_merge'] == 'right_only', colnamex] = merge.loc[merge['_merge'] == 'right_only', colnamey]

merge = merge.drop([x+'_y' for x in plft], 'columns').rename(columns=dict(zip([x+'_x' for x in plft], plft)))
print(merge.shape)

prgt = nsubcolumns[7:]

merge = merge.drop([x+'_x' for x in prgt], 'columns').rename(columns=dict(zip([x+'_y' for x in prgt], prgt)))
print(merge.shape)

(296447, 75)
(296447, 53)


In [75]:
auth = 'dos Santos Silva, Manasses; da Hora Goes, Naiala; dos Santos-Serejo, Janay Almeida; Ferreira, Claudia Fortes; Amorim, Edson Perito'
merge[merge['Author Full Names'] == auth]['Abstract'].values[0]

'The present study investigated phenolic compounds and enzymes involved in female fertility in banana plants of the Cavendish subgroup. The wild diploid Calcutta 4 and commercial cultivar Grand Naine (Cavendish subgroup) were used. The following five stages of floral development were proposed: S1 (partial vertical emission), S2 (total vertical emission), S3 (total horizontal emission), S4 (pre-anthesis), and S5 (anthesis). Following collection, pistillate (female) flowers were freeze-dried for the subsequent removal of nectaries and the analysis of phenolic compounds (PCs), antioxidant activity (DPPH and ABTS), enzymatic activity [peroxidase (POD) and polyphenol oxidase (PPO)], and total proteins (TPs). The highest values were recorded at the S3 stage, with the values decreasing as the stages progressed (until S5). At the S3 stage, the following results were obtained for Calcutta 4 and Grand Naine, respectively: PCs (32.4 and 36.1 mg GAE center dot g(-1)); DPPH (735.2 and 454.4 mu M TE

In [72]:
toint = ['Cited Reference Count', 'Open access fees (USD)', 'Times Cited, All Databases', 'Publication Year']
merge[toint] = merge[toint].apply(pd.to_numeric, downcast='integer')
merge['Keywords Plus'] = merge['Keywords Plus'].str.upper()
merge['Journal'] = merge['Journal'].str.upper()

ValueError: Unable to parse string "2,010" at position 19919

In [15]:
jnansoc = [
    'AUSTRALIAN JOURNAL OF BOTANY',
    'BIOLOGIA PLANTARUM', 
    'BOTANICAL JOURNAL OF THE LINNEAN SOCIETY', 
    'BOTANICAL STUDIES', 
    'EUPHYTICA', 
    'EUROPEAN JOURNAL OF TAXONOMY', 
    'FUNCTIONAL PLANT BIOLOGY', 
    'GENETIC RESOURCES AND CROP EVOLUTION', 
    'HORTICULTURE RESEARCH', 
    'JOURNAL OF APPLIED BOTANY AND FOOD QUALITY', 
    'JOURNAL OF APPLIED RESEARCH ON MEDICINAL AND AROMATIC PLANTS', 
    'JOURNAL OF ESSENTIAL OIL BEARING PLANTS', 
    'JOURNAL OF PHYTOPATHOLOGY',
    'JOURNAL OF PLANT BIOCHEMISTRY AND BIOTECHNOLOGY', 
    'JOURNAL OF PLANT GROWTH REGULATION', 
    'JOURNAL OF PLANT NUTRITION', 
    'JOURNAL OF PLANT PATHOLOGY', 
    'JOURNAL OF PLANT RESEARCH', 
    'MOLECULAR BREEDING', 
    'PERSPECTIVES IN PLANT ECOLOGY EVOLUTION AND SYSTEMATICS', 
    'PHOTOSYNTHETICA', 
    'PHYTOCHEMISTRY LETTERS', 
    'PLANT AND SOIL', 
    'PLANT ECOLOGY', 
    'PLANT FOODS FOR HUMAN NUTRITION',
    'PLANT GENOME', 
    'PLANT GROWTH REGULATION', 
    'PLANT SYSTEMATICS AND EVOLUTION', 
    'PLANTS-BASEL', 
    'RECORDS OF NATURAL PRODUCTS', 
    'RHIZOSPHERE', 
    'TROPICAL PLANT BIOLOGY', 
    'TURKISH JOURNAL OF BOTANY', 
    'WEED TECHNOLOGY'
]
nansoc = [
    'No',
    'Yes',
    'No',
    'Yes',
    'No',
    'No',
    'Yes',
    'No',
    'No',
    'Yes',
    'No',
    'No',
    'No',
    'No',
    'No',
    'No',
    'Yes',
    'Yes',
    'Yes',
    'No',
    'Yes',
    'Yes',
    'No',
    'No',
    'No',
    'Yes',
    'No',
    'No',
    'No',
    'No',
    'No',
    'No',
    'No',
    'Yes'
]
snansoc = [
    '',
    'Czech academy of agricultural sciences',
    '',
    'Institute of Plant and Microbial Biology',
    '',
    '',
    'Australian Society of Plant Scientists',
    '',
    '',
    'German Society for Quality Research on Plant Foods',
    '',
    '',
    '',
    '',
    '',
    '',
    'The Italian Phytopathological Society',
    'The Botanical Society of Japan',
    'National key Laboratory of Crop Genetic Improvement, HZAU',
    '',
    'Czech academy of agricultural sciences',
    'Phytochemical Society of Europe',
    '',
    '',
    '',
    'Crop Science Society of America',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    'Weed Science Society of America'
]

In [ ]:
for i in range(len(snansoc)):
    print(i,jnansoc[i],nansoc[i],snansoc[i],sep='\t')

In [24]:
#merge.loc[merge['Journal'] == 'AUSTRALIAN JOURNAL OF BOTANY','Society (Yes or No)'] = 'No'
for i in range(len(jnansoc)):
    jour = jnansoc[i]
    jsyn = nansoc[i]
    jsoc = snansoc[i]
    
    merge.loc[merge['Journal'] == jour,'Society (Yes or No)'] = jsyn
    if jsyn == 'Yes':
        merge.loc[merge['Journal'] == jour, 'Name of the society'] = jsoc

In [25]:
mjour = pd.unique(merge['Journal'])
mfields = [
    'Open access fees (USD)',
    'Name of the society',
    'Quartile',
    'Society (Yes or No)',
    'Subspecality (JCR)',
    'Open Access options available',
    'Journal location',
    'Publisher',
    'Publisher City',
    'Publisher Address',
    'ISSN',
    'eISSN'
]
for jour in mjour:
    for year in range(2000,2022,1):
        jmask = (merge['Journal'] == jour) & (merge['Publication Year'] == year)
        dfj = merge[jmask]
        for field in mfields:
            mode = dfj[field].mode()
            if len(mode) > 0:
                mode = mode[0]
                merge.loc[jmask, field] = mode

In [26]:
for field in mfields:
    foo = sorted(pd.unique(merge[pd.isnull(merge[field])]['Journal']))
    print(field, '\n', foo, '\n-----------')

Open access fees (USD) 
 ['BRYOLOGIST', 'JOURNAL OF APPLIED BOTANY AND FOOD QUALITY', 'JOURNAL OF APPLIED RESEARCH ON MEDICINAL AND AROMATIC PLANTS', 'JOURNAL OF ESSENTIAL OIL BEARING PLANTS', 'NEW ZEALAND JOURNAL OF BOTANY', 'PLANT PHYSIOLOGY AND BIOCHEMISTRY', 'PLANTA MEDICA', 'RUSSIAN JOURNAL OF PLANT PHYSIOLOGY'] 
-----------
Name of the society 
 ['AGRONOMY-BASEL', 'ANNALS OF BOTANY', 'AUSTRALIAN JOURNAL OF BOTANY', 'BMC PLANT BIOLOGY', 'BOTANICAL JOURNAL OF THE LINNEAN SOCIETY', 'BOTANY', 'CURRENT PLANT BIOLOGY', 'EUPHYTICA', 'EUROPEAN JOURNAL OF PLANT PATHOLOGY', 'EUROPEAN JOURNAL OF TAXONOMY', 'FRONTIERS IN PLANT SCIENCE', 'GENETIC RESOURCES AND CROP EVOLUTION', 'HORTICULTURAL PLANT JOURNAL', 'HORTICULTURE RESEARCH', 'INTERNATIONAL JOURNAL OF PLANT SCIENCES', 'JOURNAL OF APPLIED RESEARCH ON MEDICINAL AND AROMATIC PLANTS', 'JOURNAL OF ESSENTIAL OIL BEARING PLANTS', 'JOURNAL OF PHYTOPATHOLOGY', 'JOURNAL OF PLANT BIOCHEMISTRY AND BIOTECHNOLOGY', 'JOURNAL OF PLANT GROWTH REGULATION

In [65]:
mfields = [
    'Open Access simplified',
    'Open Access Designations'
]
for jour in mjour:
    for year in range(2000,2022,1):
        jmask = (merge['Journal'] == jour) & (merge['Publication Year'] == year)
        dfj = merge[jmask]
        for field in mfields:
            mode = dfj[field].mode()
            if len(mode) > 0:
                mode = mode[0]
                merge.loc[jmask & pd.isnull(merge[field]), field] = mode

In [69]:
for field in mfields:
    foo = sorted(pd.unique(merge[pd.isnull(merge[field])]['Journal']))
    print(field, '\n', foo, '\n-----------')

Open Access simplified 
 ['ACTA PHYSIOLOGIAE PLANTARUM', 'APPLIED VEGETATION SCIENCE', 'AUSTRALASIAN PLANT PATHOLOGY', 'AUSTRALIAN JOURNAL OF BOTANY', 'BIOLOGIA PLANTARUM', 'BOTANICAL JOURNAL OF THE LINNEAN SOCIETY', 'BOTANICAL STUDIES', 'BREEDING SCIENCE', 'EUPHYTICA', 'FUNCTIONAL PLANT BIOLOGY', 'GENETIC RESOURCES AND CROP EVOLUTION', 'JOURNAL OF APPLIED BOTANY AND FOOD QUALITY', 'JOURNAL OF ESSENTIAL OIL BEARING PLANTS', 'JOURNAL OF ETHNOPHARMACOLOGY', 'JOURNAL OF GENERAL PLANT PATHOLOGY', 'JOURNAL OF NATURAL PRODUCTS', 'JOURNAL OF PHYTOPATHOLOGY', 'JOURNAL OF PLANT BIOCHEMISTRY AND BIOTECHNOLOGY', 'JOURNAL OF PLANT BIOLOGY', 'JOURNAL OF PLANT DISEASES AND PROTECTION', 'JOURNAL OF PLANT ECOLOGY', 'JOURNAL OF PLANT GROWTH REGULATION', 'JOURNAL OF PLANT NUTRITION', 'JOURNAL OF PLANT PATHOLOGY', 'JOURNAL OF PLANT RESEARCH', 'PHOTOSYNTHETICA', 'PHYTOCHEMISTRY LETTERS', 'PHYTOPARASITICA', 'PHYTOPATHOLOGIA MEDITERRANEA', 'PLANT AND SOIL', 'PLANT BIOTECHNOLOGY', 'PLANT ECOLOGY', 'PLANT FOO

In [105]:
print(pd.unique(merge['Open access fees (USD)']))
print(pd.unique(merge['Open Access simplified']))
print(sorted(pd.unique(merge['Open Access options available'])))
print(pd.unique(merge['inferred_gender']))

[ 3400.  2250.    nan  2550.  3085.  3000.  2890.  3500.  3190.  1300.
  2010.  3390.  4000.  3255.  1800.  2040.  2750.  3800.  4200.  3280.
  2780.  2200.  2950.  1400.  1500.  3170.  1660.  2900.  2940.  3860.
  2000.  4030.  3650.  4190.  2320.  2490.  3927.   408.  3290.  3100.
  2650.  2660.  4800.  3411.  3708.  4900.  3496. 11720.  2143.  2700.
   782.  3564.  2145.   397.   990.     0.  5000.  2085.   870.  3320.
   811.  2125.   346.  2189.   750.  2970.  2500.]
['Green, Bronze, or Subscription' 'Gold' nan]
['Hybrid', 'Open', 'Subscription']
['male' 'female' nan 'unknown']


In [102]:
merge[merge['Open Access simplified'] == 'Bronze']

,Author Full Names,Article Title,Journal,impact factor (2020),Author Keywords,Keywords Plus,Abstract,Addresses--all authors,Nations--all authors,Corresponding author Addresses,...,Corresponding_author_last_name,first and last names,First_names_extracted,First_names_currated,inferred_gender,gender_inference_accuracy,#_samples_gender_inference,Per_capita_income_2020,GDP_2020,_merge
90839,"Brown, Adam O.; McNeil, Jeremy N.",Fruit production in cranberry (Ericaceae : Vac...,AMERICAN JOURNAL OF BOTANY,3.844,abortion; Ericaceae; insect; pollination; Queb...,SEED PRODUCTION; LIFE-HISTORY; LEAF REMOVAL; S...,In the cultivated cranberry (Vaccinium macroca...,"Univ Laval, Dept Biol, Ste Foy, PQ G1K 7P4, Ca...",NaN,"McNeil JN (corresponding author) Univ Laval, D...",...,McNeil,"McNeil, Jeremy N.",Jeremy,Jeremy,male,100.0,2059.0,NaN,NaN,left_only


In [68]:
merge[pd.isnull(merge['Open Access Designations'])]

,Author Full Names,Article Title,Journal,impact factor (2020),Author Keywords,Keywords Plus,Abstract,Addresses--all authors,Nations--all authors,Corresponding author Addresses,...,Corresponding_author_last_name,first and last names,First_names_extracted,First_names_currated,inferred_gender,gender_inference_accuracy,#_samples_gender_inference,Per_capita_income_2020,GDP_2020,_merge
87,"Camargo, Eduardo Caffagni; Rossi, Raquel Andra...",Comparing pulse amplitude modulated (PAM) fluo...,EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Carbon-14; chlorophyll fluorescence; electron ...,ELECTRON-TRANSPORT; PHOTOSYSTEM-II; C-14 UPTAK...,Integration of pulse amplitude modulated (PAM)...,"[Camargo, Eduardo Caffagni; Silva, Jaqueline C...",NaN,"Camargo, EC (corresponding author), Univ Fed S...",...,Camargo,"['Camargo, Eduardo Caffagni']",Eduardo Caffagni,Eduardo Caffagni,male,100.0,13431.0,14834.193700,1.450000e+12,both
334,"Wadmare, Neha; Baghela, Abhishek; Kociolek, Jo...",Description and phylogenetic position of three...,EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Molecular phylogeny; rbcL; rocky pools; Stauro...,DIATOMS BACILLARIOPHYTA; FRESH; DNA; SELLAPHOR...,Stauroneis Ehrenberg is a species-rich genus t...,"[Wadmare, Neha; Baghela, Abhishek; Karthick, B...",NaN,"Karthick, B (corresponding author), Agharkar R...",...,Karthick,"['Karthick, Balasubramanian']",Balasubramanian,Balasubramanian,male,99.0,320.0,6525.428052,2.670000e+12,both
476,"Melero-Jimenez, Ignacio J.; Martin-Clemente, E...",Dispersal of populations and environmental det...,EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Evolutionary rescue; experimental evolution; m...,NaN,Evolutionary rescue (ER) occurs when adaptatio...,"[Melero-Jimenez, Ignacio J.; Martin-Clemente, ...",NaN,"Melero-Jimenez, IJ (corresponding author), Uni...",...,Melero-Jimenez,"['Melero-Jimenez, Ignacio J.']",Ignacio J.,Ignacio J.,male,100.0,3.0,37756.351000,1.280000e+12,both
1084,"Alvarez, Claudio; Le Quesne, Carlos; Rojas-Bad...",Dendrochronological potential of Prumnopitys a...,NEW ZEALAND JOURNAL OF BOTANY,0.970,Dendroclimatology; dendroecology; Podocarpacea...,NEW-ZEALAND; TREE-RINGS; VARIABILITY; FORESTS;...,Prumnopitys andina is a South American podocar...,"[Alvarez, Claudio; Le Quesne, Carlos; Rojas-Ba...",NaN,"Alvarez, C (corresponding author), Univ Austra...",...,Alvarez,"['Alvarez, Claudio']",Claudio,Claudio,male,99.0,1560.0,25089.031300,2.530000e+11,both
1094,"Aulicino, MB; Arturi, MJ",Phenotypic diversity in Argentinian population...,NEW ZEALAND JOURNAL OF BOTANY,0.970,Bromus catharticus; genetic variation; environ...,FLORAL TRAITS; RAPHANUS-SATIVUS; BREEDING SYST...,Genetic and environmental components were anal...,"Natl Univ La Plata, Fac Ciencias Agrarias & Fo...",NaN,"Aulicino, MB (corresponding author), Benavidez...",...,Aulicino,"['Aulicino, MB']",NaN,NaN,NaN,NaN,NaN,20769.032380,3.900000e+11,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296442,"Senoussi, Mohammed Mourad; Nora, Bassa; Joel, ...",Impact of hypoxia on the growth and alkaloid a...,ACTA PHYSIOLOGIAE PLANTARUM,2.354,Catharanthus roseus; Ajmalicine; Hypoxia; Cyto...,HIGH-DENSITY CULTURES; AJMALICINE PRODUCTION; ...,The Madagascar periwinkle (Catharanthus roseus...,"[Senoussi, Mohammed Mourad; Nora, Bassa] Univ ...",NaN,"Senoussi, MM (corresponding author), Univ Larb...",...,Senoussi,"['Senoussi, Mohammed Mourad']",Mohammed Mourad,Mohammed Mourad,male,98.0,6999.0,11323.312220,1.450000e+11,right_only
296443,"Boutoumi, Hocine; Moulay, Saad; Khodja, Mohamed",Essential Oil from Ruta montana L. (Rutaceae) ...,JOURNAL OF ESSENTIAL OIL BEARING PLANTS,1.699,German cockroach; Culex pipiens; Ruta montana,TETRANYCHUS-URTICAE KOCH; RESPIRATORY ALLERGY;...,Chromatographic analysis of the essential oil ...,"[Moulay, Saad] Univ Saad Dahlab Blida, Dept Ch...",NaN,"Moulay, S (corresponding author), Univ Saad Da...",...,Moulay,"['Moulay, Saad']",Saad,Saad,male,97.0,414.0,11323.312220,1.450000e+11,right_only
296444,"Seghir

In [107]:
print(sorted(pd.unique(merge['Quartile'])))
print(sorted(pd.unique(merge['Society (Yes or No)'])))
print(sorted(pd.unique(merge['Subspecality (JCR)'])))
print(sorted(pd.unique(merge['Journal location'])))
print(sorted(pd.unique(merge['Publisher'])))
print(sorted(pd.unique(merge['Publisher City'])))
#print(sorted(pd.unique(merge['Open Access Designations'])[1:]))
pd.unique(merge['Open Access Designations'])

['Q1', 'Q2', 'Q3', 'Q4']
['No', 'Yes']
['Agronomy', 'Agronomy, Biotechnology and applied microbiology', 'Agronomy, Horticulture', 'Agronomy, Horticulture, genetics and heridity', 'Biochemistry and Molecular Biology, Biotechnology and applied microbiology', 'Biochemistry and molecular biology', 'Biochemistry and molecular biology, Cell biology', 'Biomedical', 'Biotechnology and applied microbiology', 'Cell biology', 'Chemistry', 'Ecology', 'Ecology, Forestry', 'Entomology, Zoology', 'Evolutionary biology', 'Genetics and heridity', 'Horticulture', 'Horticulture, Genetics and heridity', 'Marine and freshwater biology', 'Microbiology', 'Nutrition, Food science, Chemistry', 'Pharmacology and Pharmacy', 'Pharmacology and Pharmacy, Integrative and complementary medicine, Chemistry, Medicinal', 'Plant pathology', 'Plant science', 'Reproductive biology', 'Soil science']
['Australia', 'Austria', 'Belgium', 'Brazil', 'Canada', 'China', 'Czech Republic', 'Denmark', 'England', 'France', 'France ', 

array(['Bronze', 'Green Published', 'Green Submitted', 'Green Accepted',
       'hybrid', 'hybrid, Green Published', 'Bronze, Green Accepted', nan,
       'Bronze, Green Published', 'Bronze, Green Submitted',
       'hybrid, Green Submitted, Green Published',
       'hybrid, Green Accepted, Green Published',
       'Green Submitted, Green Accepted', 'hybrid, Green Accepted',
       'Bronze, Green Submitted, Green Published',
       'Green Submitted, Green Published',
       'hybrid, Green Submitted, Green Accepted, Green Published',
       'hybrid, Green Submitted',
       'Bronze, Green Submitted, Green Accepted', 'gold, Green Submitted',
       'gold', 'gold, Green Published',
       'gold, Green Submitted, Green Published', 'Closed',
       'gold, Green Accepted', 'gold, Green Submitted, Green Accepted',
       'Green Submitted, Green Accepted, Green Published',
       'Green Accepted, Green Published',
       'Bronze, Green Accepted, Green Published',
       'Bronze, Green Submitte

In [ ]:
jidx = 185193
for i,c in enumerate(merge.columns):
    print(i,c,merge.loc[jidx, c],sep='\t')
print('\n----------------\n')
for i,c in enumerate(col2):
    print(i,c,merge.loc[jidx, c],sep='\t')

In [103]:
mancol = ['Corresponding_author_last_name',
          'gender_inference_accuracy',
          'first and last names',
          'First_names_extracted',
          'inferred_gender',
          '#_samples_gender_inference',
          'First_names_currated',
          'Open Access simplified',
          'Open Access Designations']

idx = 0
mancolv = [
    ['Bertrand', 100, ['Bertrand, Michel'], 'Michel', 'male', 2058, 'Michel', 'Green, Bronze, or Subscription', ''],
    ['Cuvertino Santoni', 100, ['Cuvertino Santoni, Jorge'], 'Jorge', 'male', 2058, 'Jorge', 'Green, Bronze, or Subscription', ''],
    ['Morley', '', ['Morley, SE'], '', '', '', '', 'Green, Bronze, or Subscription', ''],
    ['Human', '', ['Human, L.R.D.'], '', '', '', '', 'Green, Bronze, or Subscription', ''],
    ['Eloff', '', ['Eloff, J. N.'], '', '', '', '', 'Green, Bronze, or Subscription', ''],
    ['Masoko', '', ['Masoko, P.'], '', '', '', '', 'Green, Bronze, or Subscription', ''],
    ['Pawlowski', '', ['Pawlowski, A.'], '', '', '', '', 'Green, Bronze, or Subscription', ''],
    ['Lundholm', 100, ['Lundholm, Nina'], 'Nina', 'female', 2058, 'Nina', 'Green, Bronze, or Subscription', ''],
    ['Horner', '', ['Horner, HT'], '', '', '', '', 'Green, Bronze, or Subscription', ''],
    ['Suzuki', '', ['Suzuki, M'], '', '', '', '', 'Green, Bronze, or Subscription', ''],
    ['McNeil', 100, ['McNeil, Jeremy N.'], 'Jeremy', 'male', 2059, 'Jeremy', 'Green, Bronze, or Subscription', ''],
    ['Stromvik', 100, ['Stromvik, Martina V.'], 'Martina', 'female', 2060, 'Martina', '', ''],
    ['Goldringer', 100, ['Goldringer, Isabelle'], 'Isabelle', 'female', 2040, 'Isabelle', 'Green, Bronze, or Subscription', ''],
    ['Jones', 100, ['Jones, Michelle L.'], 'Michelle', 'female', 2040, 'Michelle', 'Green, Bronze, or Subscription', 'hybrid'],
    ['Holton', 100, ['Holton, Timothy A.'], 'Timothy', 'male', 2040, 'Timothy', 'Green, Bronze, or Subscription', 'hybrid'],
    ['Chan', 100, ['Chan, Raquel L.'], 'Raquel', 'female', 2040, 'Raquel', 'Green, Bronze, or Subscription', ''],
    ['Clearwater', 100, ['Clearwater, Michael J.'], "Michael", 'male', 2020, 'Michael', 'Green, Bronze, or Subscription', 'hybrid'],
    ['Paek', 60, ['Paek, Kyung-Hee'], 'Kyung-Hee', 'female', 100, 'Kyung-Hee', 'Green, Bronze, or Subscription', ''],
    ['Woodrow', '', ['Woodrow, IE'], '', '', '','', 'Green, Bronze, or Subscription', ''],
    ['Okita', '', ['Okita, Thomas W.'], 'Thomas', 'male', 100, 'Thomas', 'Green, Bronze, or Subscription', 'Bronze'],
    ['Cho', 10, ['Cho, MJ'], 'Moo Je', 'unknown', 10, 'Moo Je', 'Green, Bronze, or Subscription', ''],
    ['Nagy', 100, ['Nagy, Ferenc'], 'Ferenc', 'male', 1000, 'Ferenc', 'Green, Bronze, or Subscription', 'Bronze']
]
len(mancolv)

22

In [104]:
for idx in range(len(lidx)):
    for fidx in range(len(mancol)):
        field = mancol[fidx]
        val = mancolv[idx][fidx]
        if val != '':
            merge.loc[lidx[idx], field] = val

In [99]:
idx = 21
for i,c in enumerate(mancol):
    print(i,c,merge.loc[lidx[idx], c],sep='\t')

0	Corresponding_author_last_name	Nagy
1	gender_inference_accuracy	100.0
2	first and last names	Nagy, Ferenc
3	First_names_extracted	Ferenc
4	inferred_gender	male
5	#_samples_gender_inference	1000.0
6	First_names_currated	Ferenc
7	Open Access simplified	Green, Bronze, or Subscription
8	Open Access Designations	Bronze


In [37]:
idx = 0
for i,c in enumerate(data.columns):
    print(i,c,merge.loc[lidx[idx], c],sep='\t')
print('\n----------------\n')
for i,c in enumerate(col2):
    print(i,c,merge.loc[lidx[idx], c],sep='\t')

0	Author Full Names	Bertrand, Michel; Monnat, Jean-Yves; Lohezic-Le Devehat, Francoise
1	Article Title	Myriolecis massei, a new species of Lecanoraceae from the coasts of the Armorican Massif in Western Europe
2	Journal	BRYOLOGIST
3	impact factor (2020)	1.2690000000000001
4	Author Keywords	Epilithic lichens; maritime; chlorinated xanthone; asemone; Brittany; Channel Islands
5	Keywords Plus	IDENTIFICATION
6	Abstract	A new species of Lecanora s.l. (Lecanoraceae) has been found in several localities on the coasts of the Armorican Massif (Brittany, Normandy, Channel Islands), only on siliceous rocks in the supralittoral zone. It is particularly distinguished by its well-developed thallus reacting C+ orange in the cortex and containing chlorinated xanthones identified by HPLC-DAD-MS. Placed in the new genus Myriolecis, it is described in this study as Myriolecis massei sp. nov. and is compared to similar maritime species containing chloroxanthones. A key is given to European maritime specie

In [108]:
filename = '../data/Plt_sci_publications_merged_8.09.csv'
merge.to_csv(filename, index=False)

# Fix abstracts

In [77]:
filename = '../data/Plt_sci_publications_geocoordinated_8.19.csv'
df = pd.read_csv(filename, index_col=False)

/home/ejam/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (26,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [78]:
print(df.shape)
print(merge.shape)

(296447, 59)
(296447, 53)


In [79]:
foo = ['Abstract', 'Funding Orgs', 'Funding Text', 'Author Keywords', 'Keywords Plus', 
       'Author Full Names', 'Article Title']
for bar in foo:
    df[bar] = merge[bar]

In [80]:
filename = '../data/Plt_sci_publications_geocoordinated_8.19.csv'
df.to_csv(filename, index=False)

In [81]:
auth = 'dos Santos Silva, Manasses; da Hora Goes, Naiala; dos Santos-Serejo, Janay Almeida; Ferreira, Claudia Fortes; Amorim, Edson Perito'
df[df['Author Full Names'] == auth]['Abstract'].values[0]

'The present study investigated phenolic compounds and enzymes involved in female fertility in banana plants of the Cavendish subgroup. The wild diploid Calcutta 4 and commercial cultivar Grand Naine (Cavendish subgroup) were used. The following five stages of floral development were proposed: S1 (partial vertical emission), S2 (total vertical emission), S3 (total horizontal emission), S4 (pre-anthesis), and S5 (anthesis). Following collection, pistillate (female) flowers were freeze-dried for the subsequent removal of nectaries and the analysis of phenolic compounds (PCs), antioxidant activity (DPPH and ABTS), enzymatic activity [peroxidase (POD) and polyphenol oxidase (PPO)], and total proteins (TPs). The highest values were recorded at the S3 stage, with the values decreasing as the stages progressed (until S5). At the S3 stage, the following results were obtained for Calcutta 4 and Grand Naine, respectively: PCs (32.4 and 36.1 mg GAE center dot g(-1)); DPPH (735.2 and 454.4 mu M TE